In [2]:
import os as os
import glob as glob
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/npatin3/miniconda/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
os.chdir('/Users/npatin3/Dropbox (GaTech)/BlueHole/Data/Metagenomes/kofamscan')

### Import the "master" KO htext file

In [5]:
ko_master = '/Users/npatin3/Dropbox (GaTech)/Workflows/KO_Orthology_ko00001.txt'

### Function to create file with KOs and number of mapped reads to each KO

In [6]:
def get_KO_readcounts(kofams, magicblast, name):
    # Drop rows with no KO
    kofams.dropna(inplace=True)
    # Select query and reference ORF from magicblast output
    mb = magicblast[['Read','ORF']]
    # Merge KO numbers with ORFs in MagicBlast results
    mb_kos = mb.merge(kofams, on='ORF', how='inner')
    # Add 'Size' column to provide count of one for each row
    mb_kos['Size'] = 1
    # Group by KO and provide sum of reads mapped to each one. Show one result for each KO only.
    mb_kos['ReadCounts'] = mb_kos.groupby(['KO'])['Size'].transform('sum')
    mb_kos.drop_duplicates(subset=['KO'], inplace=True)
    KOs_readcounts = mb_kos[['KO','ReadCounts']]
    KOs_readcounts['Sample'] = name
    return(KOs_readcounts)

In [8]:
for file in glob.glob("*kofams.txt"):
    a, b = file.split('.')
    c, d, e = a.split('_')
    name = c
    print(file, a, name)

BH37_orfs_kofams.txt BH37_orfs_kofams BH37
BH091995M_orfs_kofams.txt BH091995M_orfs_kofams BH091995M
BH091960M_orfs_kofams.txt BH091960M_orfs_kofams BH091960M
BH51_orfs_kofams.txt BH51_orfs_kofams BH51


### Get read counts for each KO

In [12]:
reads = []
for file in glob.glob("*kofams.txt"):
    kofams = pd.read_csv(file, sep='\t', names=['ORF', 'KO'])
    a, b = file.split('.')
    c, d, e = a.split('_')
    name = c
    mb = pd.read_csv('%s_magicblast_out.fltrdBstHts.blst' % name, comment='#', sep='\t', header=None)
    mb.columns = ["Read","ORF","% identity","not used","not used",
                      "not used","query start","query end","reference start",
                      "reference end","not used","not used","score",
                      "query strand","reference strand","query length",
                      "BTOP","num placements","not used","compartment",
                      "left overhang","right overhang","mate reference",
                      "mate ref. start","composite score"]
    df = get_KO_readcounts(kofams, mb, name)
    reads.append(df)

reads_df = pd.concat(reads)

/Users/npatin3/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
reads_df.head()

,KO,ReadCounts,Sample
0,K03531,753,BH37
35,K03685,335,BH37
188,K12373,916,BH37
642,K03499,1150,BH37
1054,K07277,737,BH37


### Match KOs, samples, and GE values

#### Import Genome Equivalent file and make a dict with GE of each sample

In [14]:
# Import GE files
mgn_GEs = pd.read_csv('MGN_GEs.txt', names=['Sample','GE'], sep='\t')
# Make a dict from the data frame, setting Sample as keys and reads as values
mgns_dict = dict(zip(mgn_GEs.Sample, mgn_GEs.GE))
# Map the GE of the corresponding sample onto the read counts file
reads_df['GE'] = reads_df['Sample'].map(mgns_dict)
# Make a dict from the reads df
kolist = reads_df['KO']
ko_dict = {}

In [15]:
for line in kolist:
    konumber = line.rstrip()
    ko_dict[konumber] = ''

### Match the KO number in column 3 of the master file with the dict keys

In [16]:
with open(ko_master, 'r') as file, open('kolist_full.tsv', 'w') as outfile:
    for line in file:
        X = line.rstrip().split('\t')
        konumber = X[3].split(' ')[0]
        if konumber in ko_dict:
            outfile.write(line)

### Import the file just created and reformat

In [17]:
kolist_full = pd.read_csv('kolist_full.tsv', sep='\t', header=None, names=['One','Two','Three','Four'])
kolist_full.head()

,One,Two,Three,Four
0,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K00844 HK; hexokinase [EC:2.7.1.1]
1,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K00845 glk; glucokinase [EC:2.7.1.2]
2,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],"K01810 GPI, pgi; glucose-6-phosphate isomeras..."
3,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],"K06859 pgi1; glucose-6-phosphate isomerase, a..."
4,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K15916 pgi-pmi; glucose/mannose-6-phosphate i...


In [18]:
def reformat_kolist_full (ko_df, reads_df):
    # Remove the numbered codes preceding each category
    ko_df[['One', 'Group']] = ko_df['One'].str.split(" ", n=1, expand=True)
    ko_df[['Two', 'Subgroup']] = ko_df['Two'].str.split(" ", n=1, expand=True)
    ko_df[['Three', 'Subgroup2']] = ko_df['Three'].str.split(" ", n=1, expand=True)
    ko_df[['Four', 'Function']] = ko_df['Four'].str.split(" ", n=1, expand=True)
    # Rename the column with KO numbers
    ko_df_new = ko_df.rename(columns={"Four": "KO"})
    # Select only relevant columns
    ko_df_new = ko_df_new[['Group','Subgroup','Subgroup2','KO','Function']]
    # Merge this data frame with the read counts data frame
    df_full = ko_df_new.merge(reads_df, on='KO', how='inner')
    # Normalize read counts by GE value of metagenome
    df_full['ReadCounts_Norm'] = df_full['ReadCounts'] / df_full['GE']
    return(df_full)

In [19]:
KOs_readcounts_full = reformat_kolist_full(kolist_full, reads_df)

### Split full data frame by sample, then recombine with dataset information

In [21]:
KOs_BH37 = KOs_readcounts_full[KOs_readcounts_full['Sample'] == 'BH37']
KOs_BH37.to_csv('KO_counts_BH37.csv')

In [22]:
KOs_BH51 = KOs_readcounts_full[KOs_readcounts_full['Sample'] == 'BH51']
KOs_BH51.to_csv('KO_counts_BH51.csv')

In [23]:
KOs_BH091960M = KOs_readcounts_full[KOs_readcounts_full['Sample'] == 'BH091960M']
KOs_BH091960M.to_csv('KO_counts_BH091960M.csv')

In [24]:
KOs_BH091995M = KOs_readcounts_full[KOs_readcounts_full['Sample'] == 'BH091995M']
KOs_BH091995M.to_csv('KO_counts_BH091995M.csv')

In [25]:
KOs_60Ms = KOs_readcounts_full[(KOs_readcounts_full['Sample'] == 'BH37') | 
                               (KOs_readcounts_full['Sample'] == 'BH091960M')]

In [26]:
KOs_deeps = KOs_readcounts_full[(KOs_readcounts_full['Sample'] == 'BH51') | 
                                (KOs_readcounts_full['Sample'] == 'BH091995M')]

### Concatenate the two data sets while maintaining the information as to which row is from which data set

In [27]:
concat = pd.concat([KOs_60Ms.assign(dataset='60Ms'), KOs_deeps.assign(dataset='deeps')])

In [28]:
concat.head()

,Group,Subgroup,Subgroup2,KO,Function,ReadCounts,Sample,GE,ReadCounts_Norm,dataset
14,Metabolism,Carbohydrate metabolism,Glycolysis / Gluconeogenesis [PATH:ko00010],K00845,glk; glucokinase [EC:2.7.1.2],629,BH091960M,2401.2,0.261952,60Ms
16,Metabolism,Carbohydrate metabolism,Galactose metabolism [PATH:ko00052],K00845,glk; glucokinase [EC:2.7.1.2],629,BH091960M,2401.2,0.261952,60Ms
18,Metabolism,Carbohydrate metabolism,Starch and sucrose metabolism [PATH:ko00500],K00845,glk; glucokinase [EC:2.7.1.2],629,BH091960M,2401.2,0.261952,60Ms
20,Metabolism,Carbohydrate metabolism,Amino sugar and nucleotide sugar metabolism [P...,K00845,glk; glucokinase [EC:2.7.1.2],629,BH091960M,2401.2,0.261952,60Ms
22,Metabolism,Biosynthesis of other secondary metabolites,Streptomycin biosynthesis [PATH:ko00521],K00845,glk; glucokinase [EC:2.7.1.2],629,BH091960M,2401.2,0.261952,60Ms


In [29]:
concat.to_csv('60Ms_vs_deeps_KOs_readcounts.csv', index=None)